# Training and deploying a tabular model using Vertex custom training job - Part 1

![Training pipeline](../images/custom-tabular.png)

In [3]:
import os
import pprint
import pandas as pd
import tensorflow as tf
import time

import matplotlib.pyplot as plt

from google.cloud import aiplatform as vertex_ai
from google.cloud.aiplatform_v1beta1 import types
from google.cloud import bigquery
from google.cloud import exceptions

from google.cloud.aiplatform.utils import JobClientWithOverride

from tensorflow.keras import layers
from tensorflow.keras.layers.experimental import preprocessing

from tensorflow_io import bigquery as tfio_bq

## Configure GCP settings

In [4]:
PREFIX = 'jkwst1'

shell_output=!gcloud config list --format 'value(core.project)' 2>/dev/null
PROJECT = shell_output[0]
print("Project ID: ", PROJECT)

Project ID:  jk-wst1


In [5]:
TENSORBOARD = 'projects/630263135640/locations/us-central1/tensorboards/5276565097790046208'

REGION = 'us-central1'
BQ_DATASET_NAME = f'{PREFIX}_dataset' 
BQ_TRAIN_SPLIT_NAME = 'training'
BQ_VALID_SPLIT_NAME = 'validation'
BQ_TEST_SPLIT_NAME = 'testing'
STAGING_BUCKET = f'gs://{PREFIX}-bucket'
VERTEX_SA = f'training-sa@{PROJECT}.iam.gserviceaccount.com'

## Submitting Vertex training jobs

### Display the model

`tf.keras.utils.plot_model(model, show_shapes=True, rankdir="LR")`

![Model](model.png)

### Prepare a training script

In [6]:
SCRIPT_FOLDER = 'trainer'
if tf.io.gfile.exists(SCRIPT_FOLDER):
    tf.io.gfile.rmtree(SCRIPT_FOLDER)
tf.io.gfile.mkdir(SCRIPT_FOLDER)
file_path = os.path.join(SCRIPT_FOLDER, 'train.py')

In [7]:
%%writefile {file_path}


# Copyright 2021 Google Inc. All Rights Reserved.
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#            http://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and

import hypertune
import json
import os
import re
import tensorflow as tf

from absl import app
from absl import flags
from absl import logging

from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.layers.experimental import preprocessing
from tensorflow_io import bigquery as tfio_bq

from tensorboard.plugins.hparams import api as tb_hp


FLAGS = flags.FLAGS
flags.DEFINE_integer('epochs', 3, 'Nubmer of epochs')
flags.DEFINE_integer('units', 32, 'Number units in a hidden layer')
flags.DEFINE_integer('per_replica_batch_size', 128, 'Per replica batch size')
flags.DEFINE_float('dropout_ratio', 0.5, 'Dropout ratio')
flags.DEFINE_string('training_table', None, 'Training table name')
flags.DEFINE_string('validation_table', None, 'Validationa table name')
flags.mark_flag_as_required('training_table')
flags.mark_flag_as_required('validation_table')

LOCAL_MODEL_DIR = '/tmp/saved_model'
LOCAL_TB_DIR = '/tmp/logs'
LOCAL_CHECKPOINT_DIR = '/tmp/checkpoints'
EVALUATION_FILE_NAME = 'evaluations.json'

# Define features
FEATURES = {
    "tip_bin": ("categorical", tf.int64),
    "trip_month": ("categorical", tf.int64),
    "trip_day": ("categorical", tf.int64),
    "trip_day_of_week": ("categorical", tf.int64),
    "trip_hour": ("categorical", tf.int64),
    "payment_type": ("categorical", tf.string),
    "pickup_grid": ("categorical", tf.string),
    "dropoff_grid": ("categorical", tf.string),
    "euclidean": ("numeric", tf.double),
    "trip_seconds": ("numeric", tf.int64),
    "trip_miles": ("numeric", tf.double),
}
TARGET_FEATURE_NAME = 'tip_bin'

 # Set hparams for Tensorboard and Vertex hp tuner
HP_DROPOUT = tb_hp.HParam("dropout")
HP_UNITS = tb_hp.HParam("units")
HPARAMS = [
    HP_UNITS,
    HP_DROPOUT,
]
METRICS = [
    tb_hp.Metric(
        "epoch_accuracy",
        group="validation",
        display_name="epoch accuracy"),
]
HPTUNE_METRIC = 'val_accuracy'
    

def set_job_dirs():
    """Sets job directories and hyperparameter tuning trial id
    based on env variables set by Vertex AI."""
    
    model_dir = os.getenv('AIP_MODEL_DIR', LOCAL_MODEL_DIR)
    tb_dir = os.getenv('AIP_TENSORBOARD_LOG_DIR', LOCAL_TB_DIR)
    checkpoint_dir = os.getenv('AIP_CHECKPOINT_DIR', LOCAL_CHECKPOINT_DIR)
    
    path = os.path.normpath(tb_dir)
    trial_id = re.match('^[0-9]+$', path.split(os.sep)[-2])
    if not trial_id:
        trial_id = '0'
    else:
        trial_id = trial_id[0]
    logging.info(trial_id)
    
    return model_dir, tb_dir, checkpoint_dir, trial_id


def get_bq_dataset(table_name, selected_fields, target_feature='tip_bin', batch_size=32):
    
    def _transform_row(row_dict):
        trimmed_dict = {column:
                       (tf.strings.strip(tensor) if tensor.dtype == 'string' else tensor) 
                       for (column,tensor) in row_dict.items()
                       }
        target = trimmed_dict.pop(target_feature)
        return (trimmed_dict, target)

    project_id, dataset_id, table_id = table_name.split('.')
    
    client = tfio_bq.BigQueryClient()
    parent = f'projects/{project_id}'

    read_session = client.read_session(
        parent=parent,
        project_id=project_id,
        table_id=table_id,
        dataset_id=dataset_id,
        selected_fields=selected_fields,
    )

    dataset = read_session.parallel_read_rows().map(_transform_row).batch(batch_size)
    
    return dataset


def get_category_encoding_layer(name, dataset, dtype):
    """Creates a CategoryEncoding layer for a given feature."""

    if dtype == tf.string:
      index = preprocessing.StringLookup()
    else:
      index = preprocessing.IntegerLookup()

    feature_ds = dataset.map(lambda x, y: x[name])
    index.adapt(feature_ds)
    encoder = preprocessing.CategoryEncoding(max_tokens=index.vocab_size())

    return lambda feature: encoder(index(feature))


def get_normalization_layer(name, dataset):
  """"Creates a Normalization layer for a given feature."""
  normalizer = preprocessing.Normalization()

  feature_ds = dataset.map(lambda x, y: x[name])
  normalizer.adapt(feature_ds)

  return normalizer


def create_model(dataset, input_features, units, dropout_ratio):
    """Creates a binary classifier for Chicago Taxi tip prediction task."""
    
    all_inputs = []
    encoded_features = []
    for feature_name, feature_info in input_features.items():
        col = tf.keras.Input(shape=(1,), name=feature_name, dtype=feature_info[1])
        if feature_info[0] == 'categorical':
            
            encoding_layer = get_category_encoding_layer(feature_name, 
                                                         dataset,
                                                         feature_info[1])
        else:
            encoding_layer = get_normalization_layer(feature_name,
                                                     dataset) 
        encoded_col = encoding_layer(col)
        all_inputs.append(col)
        encoded_features.append(encoded_col)
        
    all_features = tf.keras.layers.concatenate(encoded_features)
    
    x = tf.keras.layers.Dense(units, activation="relu")(all_features)
    x = tf.keras.layers.Dropout(dropout_ratio)(x)
    output = tf.keras.layers.Dense(1)(x)
    model = tf.keras.Model(all_inputs, output)
    
    return model


class HptuneCallback(tf.keras.callbacks.Callback):
    """
    A custom Keras callback class that reports a metric to hypertuner
    at the end of each epoch.
    """
    
    def __init__(self, metric_tag, metric_value):
        super(HptuneCallback, self).__init__()
        self.metric_tag = metric_tag
        self.metric_value = metric_value
        self.hpt = hypertune.HyperTune()
        
    def on_epoch_end(self, epoch, logs=None):
        self.hpt.report_hyperparameter_tuning_metric(
            hyperparameter_metric_tag=self.metric_tag,
            metric_value=logs[self.metric_value],
            global_step=epoch)
        

def main(argv):
    del argv
    
    # Set distribution strategy
    strategy = tf.distribute.MirroredStrategy()
    
    global_batch_size = (strategy.num_replicas_in_sync *
                         FLAGS.per_replica_batch_size)
    
    # Prepare datasets
    selected_fields = {key: {'output_type': value[1]} for key, value in FEATURES.items()}
    validation_ds = get_bq_dataset(FLAGS.validation_table, 
                                   selected_fields, 
                                   batch_size=global_batch_size)
    training_ds = get_bq_dataset(FLAGS.training_table,
                                 selected_fields,
                                 batch_size=global_batch_size)
    
    # Configure Tensorboard hparams
    model_dir, tb_dir, checkpoint_dir, trial_id = set_job_dirs()
    with tf.summary.create_file_writer(tb_dir).as_default():
        tb_hp.hparams_config(hparams=HPARAMS, metrics=METRICS)
        
    hparams = {
        HP_UNITS: FLAGS.units,
        HP_DROPOUT: FLAGS.dropout_ratio
    }
    
    # Create the model
    input_features = {key: value for key, value in FEATURES.items() if key != TARGET_FEATURE_NAME}
    logging.info('Creating the model ...')
    with strategy.scope():
        model = create_model(training_ds, input_features, hparams[HP_UNITS], hparams[HP_DROPOUT])
        model.compile(optimizer='adam',
                  loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
                  metrics=['accuracy'])
    
    # Configure training regimen
    callbacks = [tf.keras.callbacks.experimental.BackupAndRestore(backup_dir=checkpoint_dir)]
    callbacks.append(tf.keras.callbacks.TensorBoard(log_dir=tb_dir, 
                                                    update_freq='batch',
                                                    profile_batch=0))
    callbacks.append(tb_hp.KerasCallback(writer=tb_dir, 
                                         hparams=hparams,
                                         trial_id=trial_id))
    callbacks.append(HptuneCallback(HPTUNE_METRIC, HPTUNE_METRIC))
    
    # Start training
    logging.info('Starting training ...')
    model.fit(training_ds, 
              epochs=FLAGS.epochs, 
              validation_data=validation_ds,
              callbacks=callbacks)
    
    # Save trained model
    logging.info('Training completed. Saving the trained model to: {}'.format(model_dir))
    model.save(model_dir)  
    
    
if __name__ == '__main__':
    logging.set_verbosity(logging.INFO)
    app.run(main)

Writing trainer/train.py


### Initialize Vertex AI SDK

In [8]:
vertex_ai.init(
    project=PROJECT,
    location=REGION,
    staging_bucket=STAGING_BUCKET
)

### Configure and submit a custom Vertex job using a script

In [9]:
job_name = f'{PREFIX}_CUSTOM_SCRIPT'
base_output_dir = '{}/jobs/{}/{}'.format(STAGING_BUCKET, job_name, time.strftime("%Y%m%d_%H%M%S"))

#container_uri = 'us-docker.pkg.dev/vertex-ai/training/tf-gpu.2-4:latest'
container_uri = 'us-docker.pkg.dev/vertex-ai/training/tf-cpu.2-4:latest'
requirements = ['tensorflow-datasets==4.3.0']
args = [
    '--epochs=2', 
    '--per_replica_batch_size=128',
    '--training_table=' + f'{PROJECT}.{BQ_DATASET_NAME}.{BQ_TRAIN_SPLIT_NAME}',
    '--validation_table=' + f'{PROJECT}.{BQ_DATASET_NAME}.{BQ_TRAIN_SPLIT_NAME}',
]

machine_type = 'n1-standard-4'
#accelerator_type = 'NVIDIA_TESLA_T4'
#accelerator_count = 1

job = vertex_ai.CustomJob.from_local_script(
    display_name=job_name,
    machine_type=machine_type,
#    accelerator_type=accelerator_type,
#    accelerator_count=accelerator_count,
    script_path='trainer/train.py',
    container_uri=container_uri,
    requirements=requirements,
    args=args,
    staging_bucket=base_output_dir
)

job.run(sync=False, 
        service_account=VERTEX_SA,
        tensorboard=TENSORBOARD,
        )

INFO:google.cloud.aiplatform.utils.source_utils:Training script copied to:
gs://jkwst1-bucket/jobs/jkwst1_CUSTOM_SCRIPT/20210628_171831/aiplatform-2021-06-28-17:18:31.372-aiplatform_custom_trainer_script-0.1.tar.gz.
INFO:google.cloud.aiplatform.jobs:Creating CustomJob
INFO:google.cloud.aiplatform.jobs:CustomJob created. Resource name: projects/630263135640/locations/us-central1/customJobs/4758497209014550528
INFO:google.cloud.aiplatform.jobs:To use this CustomJob in another session:
INFO:google.cloud.aiplatform.jobs:custom_job = aiplatform.CustomJob.get('projects/630263135640/locations/us-central1/customJobs/4758497209014550528')
INFO:google.cloud.aiplatform.jobs:View Custom Job:
https://console.cloud.google.com/ai/platform/locations/us-central1/training/4758497209014550528?project=630263135640
INFO:google.cloud.aiplatform.jobs:View Tensorboard:
https://us-central1.tensorboard.googleusercontent.com/experiment/projects+630263135640+locations+us-central1+tensorboards+5276565097790046208+

### Configure and submit a Vertex job using a custom container

#### Create a docker file

In [10]:
BASE_IMAGE = 'gcr.io/deeplearning-platform-release/tf2-cpu.2-4'
TRAIN_IMAGE = f'gcr.io/{PROJECT}/{PREFIX}_taxi_trainer'

dockerfile = f'''
FROM {BASE_IMAGE}

WORKDIR /trainer
RUN pip install cloudml-hypertune

# Copies the trainer code to the docker image.
COPY train.py .

ENTRYPOINT ["python", "train.py"]
'''

with open(os.path.join(SCRIPT_FOLDER, 'Dockerfile'), 'w') as f:
    f.write(dockerfile)

#### Build a container image

In [11]:
!gcloud builds submit --tag {TRAIN_IMAGE} {SCRIPT_FOLDER}

Creating temporary tarball archive of 2 file(s) totalling 9.0 KiB before compression.
Uploading tarball of [trainer] to [gs://jk-wst1_cloudbuild/source/1624900738.281183-e25ad1a5405041cea901455a44f7268b.tgz]
Created [https://cloudbuild.googleapis.com/v1/projects/jk-wst1/locations/global/builds/74e39738-dc48-4dc0-a253-a5df8afc2200].
Logs are available at [https://console.cloud.google.com/cloud-build/builds/74e39738-dc48-4dc0-a253-a5df8afc2200?project=630263135640].
----------------------------- REMOTE BUILD OUTPUT ------------------------------
starting build "74e39738-dc48-4dc0-a253-a5df8afc2200"

FETCHSOURCE
Fetching storage object: gs://jk-wst1_cloudbuild/source/1624900738.281183-e25ad1a5405041cea901455a44f7268b.tgz#1624900739030774
Copying gs://jk-wst1_cloudbuild/source/1624900738.281183-e25ad1a5405041cea901455a44f7268b.tgz#1624900739030774...
/ [1 files][  3.4 KiB/  3.4 KiB]                                                
Operation completed over 1 objects/3.4 KiB.
INFO:google.clou

#### Prepare worker pool specification

In [14]:
worker_pool_specs =  [
    {
        "machine_spec": {
            "machine_type": "n1-standard-4",
#            "accelerator_type": "NVIDIA_TESLA_T4",
#            "accelerator_count": 1,
        },
        "replica_count": 1,
        "container_spec": {
            "image_uri": TRAIN_IMAGE,
#            "command": ["python", "train.py"],
            "args": [
                '--epochs=2', 
                '--per_replica_batch_size=128',
                '--training_table=' + f'{PROJECT}.{BQ_DATASET_NAME}.{BQ_TRAIN_SPLIT_NAME}',
                '--validation_table=' + f'{PROJECT}.{BQ_DATASET_NAME}.{BQ_VALID_SPLIT_NAME}',
            ],
        },
    }
]

INFO:google.cloud.aiplatform.jobs:CustomJob projects/630263135640/locations/us-central1/customJobs/7945919845286019072 current state:
JobState.JOB_STATE_PENDING


#### Submit and monitor the job

In [15]:
job_name = f'{PREFIX}_CUSTOM_CONTAINER'
base_output_dir = '{}/jobs/{}/{}'.format(STAGING_BUCKET, job_name, time.strftime("%Y%m%d_%H%M%S"))

job = vertex_ai.CustomJob(
    display_name=job_name,
    worker_pool_specs=worker_pool_specs,
    staging_bucket=base_output_dir
)

job.run(sync=False, 
        service_account=VERTEX_SA,
        tensorboard=TENSORBOARD
)

INFO:google.cloud.aiplatform.jobs:Creating CustomJob
INFO:google.cloud.aiplatform.jobs:CustomJob created. Resource name: projects/630263135640/locations/us-central1/customJobs/5306810463646908416
INFO:google.cloud.aiplatform.jobs:To use this CustomJob in another session:
INFO:google.cloud.aiplatform.jobs:custom_job = aiplatform.CustomJob.get('projects/630263135640/locations/us-central1/customJobs/5306810463646908416')
INFO:google.cloud.aiplatform.jobs:View Custom Job:
https://console.cloud.google.com/ai/platform/locations/us-central1/training/5306810463646908416?project=630263135640
INFO:google.cloud.aiplatform.jobs:View Tensorboard:
https://us-central1.tensorboard.googleusercontent.com/experiment/projects+630263135640+locations+us-central1+tensorboards+5276565097790046208+experiments+5306810463646908416


### Configure and submit a hyperparameter tuning job

In [16]:
job_name = f'{PREFIX}_HYPERPARAMETER'
base_output_dir = '{}/jobs/{}/{}'.format(STAGING_BUCKET, job_name, time.strftime("%Y%m%d_%H%M%S"))

hyperparameter_tuning_job_spec = {
        "display_name": job_name,
        "max_trial_count": 3,
        "parallel_trial_count": 3,
        "max_failed_trial_count": 1,
        "study_spec": {
            "metrics": [
                {
                    "metric_id": "val_accuracy",
                    "goal": vertex_ai.gapic.StudySpec.MetricSpec.GoalType.MAXIMIZE,
                }
            ],
            "parameters": [
                {
                    "parameter_id": "units",
                    "discrete_value_spec": {"values": [32, 64]},
                },
                {
                    "parameter_id": "dropout_ratio",
                    "double_value_spec": {"min_value": 0.4, "max_value": 0.6},
                }
            ],
        },
        "trial_job_spec": {
            "base_output_directory": {
                "output_uri_prefix": base_output_dir
            },
            "service_account": VERTEX_SA,
            "tensorboard": TENSORBOARD,
            "worker_pool_specs": [
                {
                    "machine_spec": {
                        "machine_type": "n1-standard-4",
                      #  "accelerator_type": vertex_ai.gapic.AcceleratorType.NVIDIA_TESLA_T4,
                      #  "accelerator_count": 1,
                    },
                    "replica_count": 1,
                    "container_spec": {
                        "image_uri": TRAIN_IMAGE,
                        "args": [
                            '--epochs=5', 
                            '--per_replica_batch_size=128',
                            '--training_table=' + f'{PROJECT}.{BQ_DATASET_NAME}.{BQ_TRAIN_SPLIT_NAME}',
                            '--validation_table=' + f'{PROJECT}.{BQ_DATASET_NAME}.{BQ_VALID_SPLIT_NAME}',
                        ],
                    },
                }
            ]
        },
    }


In [17]:
job_client = vertex_ai.initializer.global_config.create_client(
        client_class=JobClientWithOverride, location_override=REGION
)

parent = f'projects/{PROJECT}/locations/{REGION}'
version = 'v1beta1'

response = job_client.select_version(version).create_hyperparameter_tuning_job(
    parent=parent, hyperparameter_tuning_job=hyperparameter_tuning_job_spec
)

INFO:google.cloud.aiplatform.jobs:CustomJob projects/630263135640/locations/us-central1/customJobs/5306810463646908416 current state:
JobState.JOB_STATE_PENDING


#### Monitor the job

In [18]:
job_spec = job_client.get_hyperparameter_tuning_job(
    name = response.name
)
print(job_spec.state)

JobState.JOB_STATE_RUNNING
INFO:google.cloud.aiplatform.jobs:CustomJob projects/630263135640/locations/us-central1/customJobs/7945919845286019072 current state:
JobState.JOB_STATE_PENDING


#### Wait for tuning to complete

In [19]:
while True:
    job_response = job_client.get_hyperparameter_tuning_job(
        name = response.name)
    if job_response.state != vertex_ai.gapic.JobState.JOB_STATE_SUCCEEDED:
        print("Study trials have not completed:", job_response.state)
        if (job_response.state == vertex_ai.gapic.JobState.JOB_STATE_FAILED or 
           job_response.state == vertex_ai.gapic.JobState.JOB_STATE_CANCELLED):
            break
    else:
        print("Study trials have completed")
        break
    time.sleep(60)

Study trials have not completed: JobState.JOB_STATE_RUNNING
INFO:google.cloud.aiplatform.jobs:CustomJob projects/630263135640/locations/us-central1/customJobs/5306810463646908416 current state:
JobState.JOB_STATE_PENDING
INFO:google.cloud.aiplatform.jobs:CustomJob projects/630263135640/locations/us-central1/customJobs/7945919845286019072 current state:
JobState.JOB_STATE_PENDING
INFO:google.cloud.aiplatform.jobs:CustomJob projects/630263135640/locations/us-central1/customJobs/5306810463646908416 current state:
JobState.JOB_STATE_PENDING
INFO:google.cloud.aiplatform.jobs:CustomJob projects/630263135640/locations/us-central1/customJobs/7945919845286019072 current state:
JobState.JOB_STATE_PENDING
Study trials have not completed: JobState.JOB_STATE_RUNNING
INFO:google.cloud.aiplatform.jobs:CustomJob projects/630263135640/locations/us-central1/customJobs/5306810463646908416 current state:
JobState.JOB_STATE_PENDING
INFO:google.cloud.aiplatform.jobs:CustomJob projects/630263135640/locations

#### Review the results of the study

In [20]:
best_accuracy = 0.0
best_trial = None
for trial in job_response.trials:
    accuracy = float(trial.final_measurement.metrics[0].value)
    if accuracy > best_accuracy:
        best_trial = trial
        best_accuracy = accuracy
print(best_trial)        

id: "1"
state: SUCCEEDED
parameters {
  parameter_id: "dropout_ratio"
  value {
    number_value: 0.5
  }
}
parameters {
  parameter_id: "units"
  value {
    number_value: 64.0
  }
}
final_measurement {
  step_count: 4
  metrics {
    metric_id: "val_accuracy"
    value: 0.8821922540664673
  }
}
start_time {
  seconds: 1624900926
  nanos: 86728844
}
end_time {
  seconds: 1624902475
}



#### Get the best model

In [60]:
model_dir = job_response.trial_job_spec.base_output_directory.output_uri_prefix

!gsutil ls {model_dir}

gs://jk1-bucket/jobs/jk1_HYPERPARAMETER/20210622_031249/
gs://jk1-bucket/jobs/jk1_HYPERPARAMETER/20210622_031249/1/
gs://jk1-bucket/jobs/jk1_HYPERPARAMETER/20210622_031249/2/
gs://jk1-bucket/jobs/jk1_HYPERPARAMETER/20210622_031249/3/


In [61]:
best_model_dir = '{}/{}'.format(model_dir, best_trial.id)

!gsutil ls {best_model_dir}

gs://jk1-bucket/jobs/jk1_HYPERPARAMETER/20210622_031249/2/
gs://jk1-bucket/jobs/jk1_HYPERPARAMETER/20210622_031249/2/checkpoints/
gs://jk1-bucket/jobs/jk1_HYPERPARAMETER/20210622_031249/2/logs/
gs://jk1-bucket/jobs/jk1_HYPERPARAMETER/20210622_031249/2/model/
